<a href="https://colab.research.google.com/github/faisaltareque/bangla_speech_recognition/blob/main/Bengali_Automatic_Speech_Recognition_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook Bengali Speech Recognition demo is presented. [DeepSpeech2](https://arxiv.org/abs/1512.02595) is an End-to-End Speech Recognition model which is used to train ASR in Bengali Language. [Mixed precision](https://www.tensorflow.org/guide/mixed_precision) is used for faster model training and faster inference on [Tensor Core](https://www.nvidia.com/en-us/data-center/tensor-cores/) enabled Nvidia GPUs. The model is trained on [Common Voice Bangla Dataset](https://arxiv.org/abs/2206.14053) for 20 epochs. WER is 40%. Further training will result in lower WER. Follow the notebook for demo result.

In [1]:
%%capture
!gdown 1n8R9Wu6Qb6LI7TDkNbzU3ZkjcKAnYshX
!gdown 1efx6jCr00wjF5kT0luG_xvx8jGQKzL0r
!unzip demo_dataset
!rm demo_dataset.zip
!pip install jiwer

In [6]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython import display
from jiwer import wer

In [7]:
test = pd.read_csv("demo.csv")
frame_length = 200
frame_step = 80
fft_length = 256
characters = [x for x in ['।', 'ঁ', 'ং', 'ঃ', 'অ', 'আ', 'ই', 'ঈ', 'উ', 'ঋ', 'এ', 'ঐ', 'ও', 'ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ', 'জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ', '়', 'া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ', '্', 'ৎ', 'ড়', 'য়', '০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯',' ']]
char_to_num = tf.keras.layers.StringLookup(vocabulary=characters, oov_token="")
num_to_char = tf.keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True)
def wav_to_spectogram(wav_file_path):
    file = tf.io.read_file(wav_file_path)
    audio, _ = tf.audio.decode_wav(file)
    audio = tf.squeeze(audio, axis=-1)
    audio = tf.cast(audio, tf.float32)
    spectrogram = tf.signal.stft(audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.math.pow(spectrogram, 0.5)
    means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
    stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
    spectrogram = (spectrogram - means) / (stddevs + 1e-10)
    return spectrogram
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = tf.keras.backend.ctc_decode(pred, input_length=input_len, greedy=True,beam_width=4)[0][0]
    output_text = []
    for result in results:
        result = tf.strings.reduce_join(num_to_char(result)).numpy().decode("utf-8")
        output_text.append(result)
    return output_text

In [8]:
model = tf.keras.models.load_model('bengali_asr.h5',compile=False)

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


In [9]:
def asr_demo(df, index):
    if index > len(df):
        print(f'Index out of range. DataFrame range is {len(df)}.')
    target = df.iloc[index]['target']
    audio_path = os.path.join('demo_wav', df.iloc[index]['path'])
    spectogram = wav_to_spectogram(audio_path)
    spectogram = tf.expand_dims(spectogram, axis= 0, name=None)
    pred = model.predict([spectogram])
    prediction = decode_batch_predictions(pred)[0]
    wer_score = wer([target], [prediction])
    print("Target: ",target)
    print("Prediction: ",prediction)
    print('Word error rate: ', wer_score, end='\n\n')
    return audio_path

In [10]:
# Demo 1
audio_path = asr_demo(test, 0)
display.Audio(audio_path)

Target:  ইংরেজির সাথে সাথে তাদের হিন্দী ও সংস্কৃত শিক্ষা দেওয়া হতো।
Prediction:  ইংরেজির সাথসানে তাদের হিন্দি ও সংস্কৃত শিক্ষা দেওয়া হত।
Word error rate:  0.4



In [11]:
# Demo 2
audio_path = asr_demo(test, 1)
display.Audio(audio_path)

Target:  চতুর্থ সপ্তাহ থেকে অবস্থার উন্নতি হতে থাকে,কিন্তু এই উন্নতির হারের কিছু হেরফের আছে।
Prediction:  চতুর্থ শপ্তা থেকে অবস্থার উন্নতি হতে থাকে কিন্তু এই উন্নতির হারে কিছু হেরফের আছে।
Word error rate:  0.3076923076923077



In [12]:
# Demo 3
audio_path = asr_demo(test, 5)
display.Audio(audio_path)

Target:  নিষিদ্ধ আদেশ তৎক্ষণাৎ শহরের উপর চাপিয়ে দেওয়া হয়েছিল।
Prediction:  নিষিদ্ধ আদেশে তক্ষণাৎ শহরের উপর চাপিয়ে দেওয়া হয়েছিল।
Word error rate:  0.25



In [13]:
# Demo 4
audio_path = asr_demo(test, 12)
display.Audio(audio_path)

Target:  প্রাতিষ্ঠানিক শিক্ষার জন্যে ছোটবেলা থেকেই তাকে সংগ্রাম করে চলতে হয়েছে।
Prediction:  প্রাতিষ্ঠানিক শিক্ষার জন্য ছোটবেলা থেগি়ে তাকে সংগ্রাম করে চলতে হযেছে।
Word error rate:  0.3



In [14]:
# Demo 5
audio_path = asr_demo(test, 50)
display.Audio(audio_path)

Target:  এই সমিতির কয়েকটি স্থানে আদর্শ কৃষি উদ্যান ছিল।
Prediction:  এই সমিতির কয়েকটি স্থানের আদর্শ কৃষদ্দান ছিল।
Word error rate:  0.375

